## Doing Transformtion for all tables

In [ ]:
input_path = "dbfs:/mnt/bronze-parquet/SalesLT/"

In [ ]:
table_name = []
for i in dbutils.fs.ls(input_path):
  table_name.append(i.name.split("/")[0])

In [ ]:
table_name

In [ ]:
from pyspark.sql.functions import from_utc_timestamp, date_format
from pyspark.sql.types import TimestampType

for i in table_name:
  df = spark.read.format('parquet').load(input_path+i+'/'+i+'.parquet')
  if i=="Product":
    df = df.drop("ThumbNailPhoto")
  column = df.columns

  for col in column:
    if "Date" in col or "date" in col:
      df = df.withColumn(col, date_format(from_utc_timestamp(df[col].cast(TimestampType()), "UTC"), "yyyy-MM-dd"))
  output_path = '/mnt/silver/SalesLT/'+ i +'/'
  df.write.format("delta").mode("overwrite").save(output_path)
                                          